# Week Seven: Sentiment with GRUs

In this assignment, you'll convert the RNN sentiment classifier from last time into a **GRU** RNN sentiment classifier. While the small dataset and tiny vocabulary that we're using here (for speed) will limit the performance of the model, it should still do substantially better than the plain RNN.

![](http://vignette1.wikia.nocookie.net/despicableme/images/b/ba/Gru.jpg/revision/latest/scale-to-width-down/250?cb=20130711023954)

## Setup

First, let's load the data as before.

In [14]:
sst_home = '../trees'

import re
import random

# Let's do 2-way positive/negative classification instead of 5-way
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            example['label'] = easy_label_map[int(line[1])]
            if example['label'] is None:
                continue
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    random.seed(1)
    random.shuffle(data)
    return data
     
training_set = load_sst_data(sst_home + '/train.txt')
dev_set = load_sst_data(sst_home + '/dev.txt')
test_set = load_sst_data(sst_home + '/test.txt')

# Note: Unlike with k-nearest neighbors, evaluation here should be fast, and we don't need to
# trim down the dev and test sets. 

Next, we'll convert the data to index vectors.

To simplify your implementation, we'll use a fixed unrolling length of 20. In the conversion process, we'll cut off excess words (towards the left/start end of the sentence), pad short sentences (to the left) with a special word symbol `<PAD>`, and mark out-of-vocabulary words with `<UNK>`, for unknown. As in the previous assignment, we'll use a very small vocabulary for this assignment, so you'll see `<UNK>` often.

In [15]:
import collections
import numpy as np

def sentence_to_padded_index_sequence(datasets):
    '''Annotates datasets with feature vectors.'''
    

    PADDING = "<PAD>"
    UNKNOWN = "<UNK>"
    SEQ_LEN = 20
    
    # Extract vocabulary
    def tokenize(string):
        return string.lower().split()
    
    word_counter = collections.Counter()
    for example in datasets[0]:
        word_counter.update(tokenize(example['text']))
    
    vocabulary = set([word for word in word_counter if word_counter[word] > 10])
    vocabulary = list(vocabulary)
    vocabulary = [PADDING, UNKNOWN] + vocabulary
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))
    indices_to_words = {v: k for k, v in word_indices.items()}
        
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['index_sequence'] = np.zeros((SEQ_LEN), dtype=np.int32)
            
            token_sequence = tokenize(example['text'])
            padding = SEQ_LEN - len(token_sequence)
            
            for i in range(SEQ_LEN):
                if i >= padding:
                    if token_sequence[i - padding] in word_indices:
                        index = word_indices[token_sequence[i - padding]]
                    else:
                        index = word_indices[UNKNOWN]
                else:
                    index = word_indices[PADDING]
                example['index_sequence'][i] = index
    return indices_to_words, word_indices
    
indices_to_words, word_indices = sentence_to_padded_index_sequence([training_set, dev_set, test_set])

In [16]:
print training_set[18]
print len(word_indices)

{'text': 'Pretty good little movie .', 'index_sequence': array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 195, 351, 487, 732, 819], dtype=int32), 'label': 1}
1250


In [17]:
def evaluate_classifier(classifier, eval_set):
    correct = 0
    hypotheses = classifier(eval_set)
    for i, example in enumerate(eval_set):
        hypothesis = hypotheses[i]
        if hypothesis == example['label']:
            correct += 1        
    return correct / float(len(eval_set))

## Building the RNN

Modify the class below to convert it into a GRU model. You should have to:

- Add additional trained parameters.
- Modify the `step()` function.
- Modify L2 regularization to incorporate the new parameters.

You shouldn't have to edit anything outside of `__init__()`.

In [18]:
import tensorflow as tf

In [46]:
class RNNSentimentClassifier:
    def __init__(self, vocab_size, sequence_length):
        # Define the hyperparameters
        self.learning_rate = 1.0  # Should be about right
        self.training_epochs = 500  # How long to train for - chosen to fit within class time
        self.display_epoch_freq = 5  # How often to test and print out statistics
        self.dim = 12  # The dimension of the hidden state of the RNN
        self.embedding_dim = 8  # The dimension of the learned word embeddings
        self.batch_size = 256  # Somewhat arbitrary - can be tuned, but often tune for speed, not accuracy
        self.vocab_size = vocab_size  # Defined by the file reader above
        self.sequence_length = sequence_length  # Defined by the file reader above
        self.l2_lambda = 0.001
        
        # Define the parameters
        self.E = tf.Variable(tf.random_normal([self.vocab_size, self.embedding_dim], stddev=0.1))
        
        self.W_rnn = tf.Variable(tf.random_normal([self.embedding_dim + self.dim, self.dim], stddev=0.1))
        self.b_rnn = tf.Variable(tf.random_normal([self.dim], stddev=0.1))
        
        self.W_r = tf.Variable(tf.random_normal([self.embedding_dim + self.dim, self.dim], stddev=0.1))
        self.b_r = tf.Variable(tf.random_normal([self.dim], stddev=0.1))
        
        self.W_z = tf.Variable(tf.random_normal([self.embedding_dim + self.dim, self.dim], stddev=0.1))
        self.b_z = tf.Variable(tf.random_normal([self.dim], stddev=0.1))
        
        self.W_cl = tf.Variable(tf.random_normal([self.dim, 2], stddev=0.1))
        self.b_cl = tf.Variable(tf.random_normal([2], stddev=0.1))
        
        # Define the placeholders
        self.x = tf.placeholder(tf.int32, [None, self.sequence_length])
        self.y = tf.placeholder(tf.int32, [None])
        
        # Define one step of the RNN
        def step(x, h_prev):
            emb = tf.nn.embedding_lookup(self.E, x)
            emb_h_prev = tf.concat(1, [emb, h_prev])
            z = tf.nn.sigmoid(tf.matmul(emb_h_prev, self.W_z)  + self.b_z)
            r = tf.nn.sigmoid(tf.matmul(emb_h_prev, self.W_r)  + self.b_r)
            emb_r_h_prev = tf.concat(1, [emb, r * h_prev])
            h_tilde = tf.nn.tanh(tf.matmul(emb_r_h_prev, self.W_rnn)  + self.b_rnn)
            h = (1. - z) * h_prev + z * h_tilde
            return h

        # Split up the inputs into individual tensors
        self.x_slices = tf.split(1, self.sequence_length, self.x)
        
        self.h_zero = tf.zeros([self.batch_size, self.dim])
        
        h_prev = self.h_zero
        
        # Unroll the RNN
        for t in range(self.sequence_length):
            x_t = tf.reshape(self.x_slices[t], [-1])
            h_prev = step(x_t, h_prev)
        
        # Compute the logits using one last linear layer
        self.logits = tf.matmul(h_prev, self.W_cl) + self.b_cl

        # Define the L2 cost
        self.l2_cost = self.l2_lambda * (tf.reduce_sum(tf.square(self.W_rnn)) +
                                         tf.reduce_sum(tf.square(self.W_z)) + 
                                         tf.reduce_sum(tf.square(self.W_r)) + 
                                         tf.reduce_sum(tf.square(self.W_cl)))
        
        # Define the cost function (here, the softmax exp and sum are built in)
        self.total_cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(self.logits, self.y)) \
            + self.l2_cost
        
        # This  performs the main SGD update equation with gradient clipping
        optimizer_obj = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate)
        gvs = optimizer_obj.compute_gradients(self.total_cost)
        capped_gvs = [(tf.clip_by_norm(grad, 5.0), var) for grad, var in gvs if grad is not None]
        self.optimizer = optimizer_obj.apply_gradients(capped_gvs)
        
        # Create an operation to fill zero values in for W and b
        self.init = tf.initialize_all_variables()
        
        # Create a placeholder for the session that will be shared between training and evaluation
        self.sess = None
        
    def train(self, training_data, dev_set):
        def get_minibatch(dataset, start_index, end_index):
            indices = range(start_index, end_index)
            vectors = np.vstack([dataset[i]['index_sequence'] for i in indices])
            labels = [dataset[i]['label'] for i in indices]
            return vectors, labels
        
        self.sess = tf.Session()
        
        self.sess.run(self.init)
        print 'Training.'

        # Training cycle
        for epoch in range(self.training_epochs):
            random.shuffle(training_set)
            avg_cost = 0.
            total_batch = int(len(training_set) / self.batch_size)
            
            # Loop over all batches in epoch
            for i in range(total_batch):
                # Assemble a minibatch of the next B examples
                minibatch_vectors, minibatch_labels = get_minibatch(training_set, 
                                                                    self.batch_size * i, 
                                                                    self.batch_size * (i + 1))

                # Run the optimizer to take a gradient step, and also fetch the value of the 
                # cost function for logging
                _, c = self.sess.run([self.optimizer, self.total_cost], 
                                     feed_dict={self.x: minibatch_vectors,
                                                self.y: minibatch_labels})
                                                                    
                # Compute average loss
                avg_cost += c / (total_batch * self.batch_size)
                
            # Display some statistics about the step
            # Evaluating only one batch worth of data -- simplifies implementation slightly
            if (epoch+1) % self.display_epoch_freq == 0:
                print "Epoch:", (epoch+1), "Cost:", avg_cost, \
                    "Dev acc:", evaluate_classifier(self.classify, dev_set[0:256]), \
                    "Train acc:", evaluate_classifier(self.classify, training_set[0:256])  
    
    def classify(self, examples):
        # This classifies a list of examples
        vectors = np.vstack([example['index_sequence'] for example in examples])
        logits = self.sess.run(self.logits, feed_dict={self.x: vectors})
        return np.argmax(logits, axis=1)

Now let's train it. If the GRU is doing what it should, you should reach 74% accuracy within your first 200 epochs—a substantial improvement over the 70% figure we saw last week.

In [47]:
classifier = RNNSentimentClassifier(len(word_indices), 20)
classifier.train(training_set, dev_set)

Training.
Epoch: 5 Cost: 0.00272594029256 Dev acc: 0.515625 Train acc: 0.578125
Epoch: 10 Cost: 0.00271705439728 Dev acc: 0.484375 Train acc: 0.52734375
Epoch: 15 Cost: 0.0027131375391 Dev acc: 0.515625 Train acc: 0.48828125
Epoch: 20 Cost: 0.00270917687427 Dev acc: 0.515625 Train acc: 0.53515625
Epoch: 25 Cost: 0.00271003589 Dev acc: 0.515625 Train acc: 0.5078125
Epoch: 30 Cost: 0.00270958790659 Dev acc: 0.515625 Train acc: 0.51171875
Epoch: 35 Cost: 0.00270593757914 Dev acc: 0.515625 Train acc: 0.49609375
Epoch: 40 Cost: 0.00270748297098 Dev acc: 0.515625 Train acc: 0.5078125
Epoch: 45 Cost: 0.00270647911734 Dev acc: 0.51953125 Train acc: 0.5390625
Epoch: 50 Cost: 0.00270740824959 Dev acc: 0.515625 Train acc: 0.5390625
Epoch: 55 Cost: 0.0027074857132 Dev acc: 0.5078125 Train acc: 0.625
Epoch: 60 Cost: 0.00269979731766 Dev acc: 0.5234375 Train acc: 0.5234375
Epoch: 65 Cost: 0.00270135373877 Dev acc: 0.50390625 Train acc: 0.5859375
Epoch: 70 Cost: 0.00268661305826 Dev acc: 0.50390625 T

## Bonus: Depth

If you finish early, try converting the GRU RNN to a *deep* GRU RNN with two 12-dimensional recurrent layers. This will require still more parameters, a more complex step function, and a few other miscellaneous changes within `__init__`.